*Хакатон Норникель - интеллектуальные горизонты. Команда PlatinumPandas*  
Состав команды:  
Иван Яковлев  
Алексей Жданов  

# Задача 1. Флотомашина времени. Решение от команды PlatinumPandas.

## Введение
**Цель работы**:  
На основании исторических данных установить диапазоны работы автомата на каждом аппарате.

**Задачи, решаемые в работе:**
1. Обработка предоставленных данных.
2. Исследовательский и корреляционный анализ данных.
3. Формирование моделей для установление диапазонов.
4. Формирование выводов по работе.

**План работы:**
1. Загрузить предоставленные данные.
2. Устранить или заполнить пропуски.
3. Выполнить исследовательский анализ данных.
4. Выполнить корреляционный анализ данных.
5. Выделить признаки для обучения и прогнозирования.
6. Сформировать модели и обучить их на исторических данных.
7. Сделать выводы по работе.

**Описание задачи и работы:**  
Организаторами Хакатона предоставлена следующая информация:
1. Датасет df_hack_final из 130 признаков, содержащий информацию о работе линии флотации действующего производства по обогащению руды.
2. Датасет с пустыми значениями, которые необходимо заполнить на основе разработанного решения.
3. Описание задания.
4. Схема установки флотации.

Формат имен признаков в датасете df_hack_final:  
- Ni_rec – извлечение никеля в готовый никелевый продукт, концентрат (значение может отсутствовать, валидны только меньше 1 и больше 0),
- Ore, oreth – имена, которые означают признаки рудного сгустителя (Ore thickener) и на входе первой ФМ (напомним, ФМ – флотомашина),
- resth – имена, которые означают признаки сгустителя с готовым никелевым продуктом (Final Ni thickener).  

Физические характеристики: Mass (масса), Dens (плотность), Vol (объём), Ni (никель), Cu (медь), A (флаг автоуправления, если оптимизатор управления ФМ включен, то равен 1 – фактические диапазоны на этой ФМ актуальны, в противном случае диапазонам не следует доверять, так как не обновлялись после выключения оптимизатора).  
Суффиксы имён ФМ (положения в цепи агрегатов): 1.1, 1.2, …, 6.2.
Суффиксы продуктов: F – питание ФМ, C – концентрат ФМ, T – хвосты ФМ.
Суффиксы границ фактических диапазонов, которые были выставлены технологом для оптимизатора ФМ: min, max.
Границы фактического диапазона в файле исходных данных – значения, которые выставляли технологи производства для сервиса оптимизации флотации (по одному на каждую линию флотомашины, например, оптимизатор для ФМ1.1.), чтобы этот оптимизатор генерировал воздействия на рычаги управления флотацией на ФМ, которые обеспечат сходимость к середине этого диапазона.  
Технологические ограничения, которым должно соответствовать решение:   
Ограничение 1. Каждый диапазон (признаки min, max) можно изменить не чаще 1 раза в 2 часа (не менее 8 15-минуток подряд с одной и той же парой значений границ).  
Ограничение 2. Наименьшие допустимые приращения признаков min, max зависят от продукта, металла и ФМ (и одинаковы для двух линий одной ФМ). В качестве приращений используйте только кратные им значения (например, если наименьшее допустимое – 0.1, используйте 0.1, 0.2, 0.3, …):




## Загрузка библиотек

In [177]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [178]:
!pip install phik -q

In [179]:
# Библиотеки для работы с таблицами
import pandas as pd
import numpy as np

# Библиотеки для построения графиков
import matplotlib.pyplot as plt
import seaborn as sns

# Библиотеки для анализа данных
from statsmodels.tsa.seasonal import seasonal_decompose
import phik
from phik.report import plot_correlation_matrix
from phik import report
import plotly.express as px

# Библиотеки для подготовки признаков
from sklearn.preprocessing import (StandardScaler)

# Библиотеки для контроля качества моделей
from sklearn.metrics import r2_score, root_mean_squared_error

# Библиотеки для работы с моделями
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor


## Подготовку функций

Ниже приведены функции для обработки значений, когда был выключен автомат.

In [180]:
def corr_range_Ni_1_1_c (df):
  value = 0
  if df['FM_1.1_A'] == 0:
   value = df['Ni_1.1C']
  return value

In [181]:
def corr_range_Ni_1_2_C (df):
  value = 0
  if df['FM_1.2_A'] == 0:
   value = df['Ni_1.2C']
  return value

In [182]:
def corr_range_Cu_3_2_C (df):
  value = 0
  if df['FM_3.2_A'] == 0:
   value = df['Cu_3.2C']
  return value

In [183]:
def corr_range_Cu_3_2_T (df):
  value = 0
  if df['FM_3.2_A'] == 0:
   value = df['Cu_3.2T']
  return value

In [184]:
def corr_range_Cu_3_1_T (df):
  value = 0
  if df['FM_3.1_A'] == 0:
   value = df['Cu_3.1T']
  return value

In [185]:
def corr_range_Cu_3_1_C (df):
  value = 0
  if df['FM_3.1_A'] == 0:
   value = df['Cu_3.1C']
  return value

In [186]:
def corr_range_Cu_2_2_C (df):
  value = 0
  if df['FM_2.2_A'] == 0:
   value = df['Cu_2.2C']
  return value

In [187]:
def corr_range_Cu_2_2_T (df):
  value = 0
  if df['FM_2.2_A'] == 0:
   value = df['Cu_2.2T']
  return value

In [188]:
def corr_range_Cu_2_1_T (df):
  value = 0
  if df['FM_2.1_A'] == 0:
   value = df['Cu_2.1T']
  return value

In [189]:
def corr_range_Cu_2_1_C (df):
  value = 0
  if df['FM_2.1_A'] == 0:
   value = df['Cu_2.1C']
  return value

In [190]:
def corr_range_Cu_1_1_C (df):
  value = 0
  if df['FM_1.1_A'] == 0:
   value = df['Cu_1.1C']
  return value

In [191]:
def corr_range_Cu_1_2_C (df):
  value = 0
  if df['FM_1.1_A'] == 0:
   value = df['Cu_1.2C']
  return value

In [192]:
def corr_range_Ni_6_1_T (df):
  value = 0
  if df['FM_6.1_A'] == 0:
   value = df['Ni_6.1T']
  return value

In [193]:
def corr_range_Ni_6_2_T (df):
  value = 0
  if df['FM_6.2_A'] == 0:
   value = df['Ni_6.2T']
  return value

In [194]:
def corr_range_Ni_6_2_C (df):
  value = 0
  if df['FM_6.2_A'] == 0:
   value = df['Ni_6.2C']
  return value

In [195]:
def corr_range_Ni_6_1_C (df):
  value = 0
  if df['FM_6.1_A'] == 0:
   value = df['Ni_6.1C']
  return value

In [196]:
def corr_range_Ni_5_1_T (df):
  value = 0
  if df['FM_5.1_A'] == 0:
   value = df['Ni_5.1T']
  return value

In [197]:
def corr_range_Ni_5_2_T (df):
  value = 0
  if df['FM_5.2_A'] == 0:
   value = df['Ni_5.2T']
  return value

In [198]:
def corr_range_Ni_5_2_C (df):
  value = 0
  if df['FM_5.2_A'] == 0:
   value = df['Ni_5.2C']
  return value

In [199]:
def corr_range_Ni_5_1_C (df):
  value = 0
  if df['FM_5.1_A'] == 0:
   value = df['Ni_5.1C']
  return value

In [200]:
def corr_range_Ni_4_1_T (df):
  value = 0
  if df['FM_4.1_A'] == 0:
   value = df['Ni_4.1T']
  return value

In [201]:
def corr_range_Ni_4_2_T (df):
  value = 0
  if df['FM_4.2_A'] == 0:
   value = df['Ni_4.2T']
  return value

In [202]:
def corr_range_Ni_4_2_C (df):
  value = 0
  if df['FM_4.2_A'] == 0:
   value = df['Ni_4.2C']
  return value

In [203]:
def corr_range_Ni_4_1_C (df):
  value = 0
  if df['FM_4.1_A'] == 0:
   value = df['Ni_4.1C']
  return value

In [204]:
# Функция обработки нулевых значений
def delete_zero(df):
  value = df
  if df < 0:
    value = 0
  return value

## Загрузка данных

Сохраним каждый датасет в отдельную переменную. Для удобства работы тестовый датасет сохраним два раза в разные переменные (где индексы это даты и где индексы это числа).

In [205]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Норникель/df_hack_final.csv', index_col='MEAS_DT', parse_dates=['MEAS_DT'])

In [206]:
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Норникель/test.csv', index_col='MEAS_DT', parse_dates=['MEAS_DT'])

In [207]:
df_test_index = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Норникель/test.csv')

Проверим корректность отображения

In [208]:
df.head()

,Cu_oreth,Ni_oreth,Ore_mass,Mass_1,Mass_2,Dens_4,Mass_4,Vol_4,Cu_4F,Ni_4F,...,Cu_3.1T_max,Cu_3.1T_min,FM_3.2_A,Cu_3.2C_max,Cu_3.2C_min,Ni_3.2C_max,Ni_3.2C_min,Cu_3.2T_max,Cu_3.2T_min,Ni_rec
MEAS_DT,,,,,,,,,,,,,,,,,,,,,
2024-01-01 00:00:00,2.6097,1.5313,1096.5,1240.597656,692.090942,1.342155,711.999023,1548.71875,0.6232,2.4937,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 00:15:00,2.5548,1.4842,1123.0,1205.422363,693.616394,1.339809,710.697815,1556.56250,0.6292,2.5157,...,1.0,0.8,0.0,14.0,12.0,3.7,3.5,1.2,1.0,NaN
2024-01-01 00:30:00,2.5109,1.4355,840.0,1188.762573,698.350586,1.339792,707.198547,1548.09375,0.5941,2.5253,...,1.0,0.8,0.0,14.0,12.0,3.7,3.5,1.2,1.0,0.970170
2024-01-01 00:45:00,2.4765,1.3852,824.0,1151.888672,714.678101,1.342392,707.865540,1538.87500,0.6682,2.5418,...,1.0,0.8,0.0,14.0,12.0,3.7,3.5,1.2,1.0,0.968639
2024-01-01 01:00:00,2.3585,1.3368,0.0,1104.101318,730.190674,1.337608,700.935059,1545.18750,0.6489,2.5559,...,1.0,0.8,0.0,14.0,12.0,3.7,3.5,1.2,1.0,0.974205


In [209]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 30336 entries, 2024-01-01 00:00:00 to 2024-11-11 23:45:00
Columns: 129 entries, Cu_oreth to Ni_rec
dtypes: float64(129)
memory usage: 30.1 MB


In [210]:
df_test.head()

,Ni_1.1C_min,Ni_1.1C_max,Cu_1.1C_min,Cu_1.1C_max,Ni_1.2C_min,Ni_1.2C_max,Cu_1.2C_min,Cu_1.2C_max,Cu_2.1T_min,Cu_2.1T_max,...,Ni_5.2C_min,Ni_5.2C_max,Ni_6.1T_min,Ni_6.1T_max,Ni_6.1C_min,Ni_6.1C_max,Ni_6.2T_min,Ni_6.2T_max,Ni_6.2C_min,Ni_6.2C_max
MEAS_DT,,,,,,,,,,,,,,,,,,,,,
2024-01-19 12:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-19 12:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-19 12:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-19 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-19 13:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [211]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6740 entries, 2024-01-19 12:15:00 to 2024-11-05 18:30:00
Data columns (total 40 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Ni_1.1C_min  0 non-null      float64
 1   Ni_1.1C_max  0 non-null      float64
 2   Cu_1.1C_min  0 non-null      float64
 3   Cu_1.1C_max  0 non-null      float64
 4   Ni_1.2C_min  0 non-null      float64
 5   Ni_1.2C_max  0 non-null      float64
 6   Cu_1.2C_min  0 non-null      float64
 7   Cu_1.2C_max  0 non-null      float64
 8   Cu_2.1T_min  0 non-null      float64
 9   Cu_2.1T_max  0 non-null      float64
 10  Cu_2.2T_min  0 non-null      float64
 11  Cu_2.2T_max  0 non-null      float64
 12  Cu_3.1T_min  0 non-null      float64
 13  Cu_3.1T_max  0 non-null      float64
 14  Cu_3.2T_min  0 non-null      float64
 15  Cu_3.2T_max  0 non-null      float64
 16  Ni_4.1T_min  0 non-null      float64
 17  Ni_4.1T_max  0 non-null      float64
 18  Ni_4.1C_min 

In [212]:
df_test_index.head()

,MEAS_DT,Ni_1.1C_min,Ni_1.1C_max,Cu_1.1C_min,Cu_1.1C_max,Ni_1.2C_min,Ni_1.2C_max,Cu_1.2C_min,Cu_1.2C_max,Cu_2.1T_min,...,Ni_5.2C_min,Ni_5.2C_max,Ni_6.1T_min,Ni_6.1T_max,Ni_6.1C_min,Ni_6.1C_max,Ni_6.2T_min,Ni_6.2T_max,Ni_6.2C_min,Ni_6.2C_max
0,2024-01-19 12:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-01-19 12:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-01-19 12:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-01-19 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-01-19 13:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [213]:
df_test_index.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6740 entries, 0 to 6739
Data columns (total 41 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MEAS_DT      6740 non-null   object 
 1   Ni_1.1C_min  0 non-null      float64
 2   Ni_1.1C_max  0 non-null      float64
 3   Cu_1.1C_min  0 non-null      float64
 4   Cu_1.1C_max  0 non-null      float64
 5   Ni_1.2C_min  0 non-null      float64
 6   Ni_1.2C_max  0 non-null      float64
 7   Cu_1.2C_min  0 non-null      float64
 8   Cu_1.2C_max  0 non-null      float64
 9   Cu_2.1T_min  0 non-null      float64
 10  Cu_2.1T_max  0 non-null      float64
 11  Cu_2.2T_min  0 non-null      float64
 12  Cu_2.2T_max  0 non-null      float64
 13  Cu_3.1T_min  0 non-null      float64
 14  Cu_3.1T_max  0 non-null      float64
 15  Cu_3.2T_min  0 non-null      float64
 16  Cu_3.2T_max  0 non-null      float64
 17  Ni_4.1T_min  0 non-null      float64
 18  Ni_4.1T_max  0 non-null      float64
 19  Ni_4.1

Данные загружены и отображаются корректно.

Отдельно подготовим датафрейм с ресемплированием по часу, чтобы можно было провести почасовой анализ работы агрегатов. В качестве функции при ресемплированием возьмем среднее.

In [214]:
df_hour = df.resample('1H').mean()

<ipython-input-214-c7c1ff4b9628>:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_hour = df.resample('1H').mean()


Заполним пропуски в данных медианой

In [215]:
for column in df_hour.columns:
  df_hour[column] = df_hour[column].fillna(df_hour[column].median())

Подготовим датафрейм с описанием каждого столбца. Для этого будем использовать основную таблицу без ресемплирования для более точных данных. Предварительно рассортируем данные в основном датафрейме.

In [216]:
df = df.reindex(sorted(df.columns), axis=1)

In [217]:
describe = df['Cu_1.1C_max'].describe()
describe = pd.DataFrame(describe)

In [218]:
for column in df.drop('Cu_1.1C_max', axis=1).columns:
  describe = describe.join(df[column].describe())


Проверим результат работы

In [219]:
describe

,Cu_1.1C_max,Cu_1.1C,Cu_1.1C_min,Cu_1.2C,Cu_1.2C_max,Cu_1.2C_min,Cu_2.1C,Cu_2.1C_max,Cu_2.1C_min,Cu_2.1T,...,Ni_6.2T_max,Ni_6.2T_min,Ni_6F,Ni_oreth,Ni_rec,Ni_resth,Ore_mass,Vol_4,Vol_5,Vol_6
count,30335.000000,24320.000000,30335.000000,28888.000000,30335.000000,30335.000000,27430.000000,30335.000000,30335.000000,24649.000000,...,30335.000000,30335.000000,27169.000000,26213.000000,27759.000000,26417.000000,30336.000000,30336.000000,30336.000000,30336.000000
mean,4.902321,4.585501,4.354795,4.927762,5.196499,4.636875,16.966245,17.777439,15.927147,0.368490,...,1.458376,1.270134,9.241147,1.685584,0.947898,9.074016,1213.996028,1520.387530,996.859847,659.749020
std,0.086483,0.231957,0.175821,0.376766,0.208677,0.172663,1.020794,1.038401,0.374704,0.068571,...,0.113729,0.141408,0.317268,0.163395,0.020704,0.382616,304.202121,254.858332,172.476622,124.849016
min,4.300000,3.702000,4.000000,3.381100,4.800000,4.400000,11.483800,17.000000,14.000000,0.247400,...,1.100000,1.000000,5.557250,1.141700,0.594336,4.346200,0.000000,0.000000,0.000000,0.000000
25%,4.900000,4.463800,4.200000,4.643775,5.200000,4.500000,16.307000,17.000000,16.000000,0.316800,...,1.450000,1.150000,9.033650,1.574000,0.933853,8.879200,1133.500000,1523.531250,996.000000,634.375000
50%,4.900000,4.599100,4.400000,4.921000,5.200000,4.600000,16.879900,17.200000,16.000000,0.345500,...,1.500000,1.300000,9.245200,1.685100,0.943993,9.118000,1274.500000,1562.812500,1026.562500,676.593750
75%,4.900000,4.740800,4.500000,5.189725,5.200000,4.800000,17.506600,18.000000,16.000000,0.433900,...,1.550000,1.400000,9.443850,1.792700,0.966216,9.315000,1395.500000,1595.750000,1052.750000,716.000000
max,5.400000,6.045000,4.700000,9.078100,5.600000,4.900000,20.422800,20.000000,16.000000,2.011300,...,1.850000,1.650000,11.698300,2.617300,1.000000,10.793200,2000.000000,2028.375000,1670.062500,1000.031250


Теперь подготовим датафрейм без ресемплирования, но с заполненными пропусками. Пропуски будем заполнять медианой.

In [220]:
df_fill_median = df

In [221]:
for column in df_fill_median.columns:
  df_fill_median[column] = df_fill_median[column].fillna(df_fill_median[column].median())

Нужные датафреймы подготовлены.

## Анализ данных

Посмотрим на распределения данных, ресемплированных по часу.

In [222]:
for column in df_hour.columns:
  df_hour[column].hist(bins=30)
  plt.title(column)
  plt.xlabel('Значение')
  plt.ylabel('Количество')
  plt.show()

Output hidden; open in https://colab.research.google.com to view.

Из анализа данных видно, что есть порядка немало случаев, когда вход и выход с линии был равен нулю. Также есть в некоторых столбцах аномальные низкие и аномально высокие значения.

Теперь проведем анализ трендов и сезонности

In [223]:
for column in df_hour.columns:
  decomposed = seasonal_decompose(df_hour[column])
  plt.figure(figsize=(10, 10))
  plt.subplot(311)
  decomposed.trend.plot(ax=plt.gca())
  plt.title(f'Тренд {column}')
  plt.xlabel('Дата')
  plt.ylabel('Значение')
  plt.subplot(312)
  decomposed.seasonal['2024-01-01':'2024-01-10'].plot(ax=plt.gca())
  plt.title(f'Сезонность {column}')
  plt.xlabel('Дата')
  plt.ylabel('Значение')
  plt.subplot(313)
  decomposed.resid.plot(ax=plt.gca())
  plt.title(f'Остатки {column}')
  plt.xlabel('Дата')
  plt.ylabel('Значение')
  plt.tight_layout()
  plt.show()

Output hidden; open in https://colab.research.google.com to view.

Из данных видно, что ни вход ни выход сырья не имеют сезонного тренда в течении года. Вместе с тем, загруженность работы аппаратов одинакова каждый день. Каждодневные графики похожи один на другой и демострируют полное совпадение. Это говорит о ритмичности производства.

Теперь проведением корреляционный анализ данных. Для этого подготовим перечень столбцов по каждому аппарату отдельно.

In [224]:
column_1_1 = ['Cu_1.1C_max', 'Cu_1.1C_min', 'Ni_1.1C_min', 'Ni_1.1C_max', 'Cu_1.1C', 'Ni_1.1C', 'Ni_1.1T_max', 'Ni_1.1T_min', 'Ni_rec', 'Ni_oreth', 'Ore_mass', 'Mass_1', 'Cu_resth', 'Ni_resth', 'Cu_oreth']
column_1_2 = ['Cu_1.2C_max', 'Cu_1.2C_min', 'Ni_1.2C_min', 'Ni_1.2C_max', 'Cu_1.2C', 'Ni_1.2C', 'Ni_1.2T_max', 'Ni_1.2T_min', 'Ni_rec', 'Ni_oreth', 'Ore_mass', 'Mass_1', 'Cu_resth', 'Ni_resth', 'Cu_oreth']
column_2_1 = ['Cu_oreth', 'Ni_oreth', 'Ore_mass', 'Mass_2', 'Cu_resth', 'Ni_resth','Cu_2F', 'Ni_2F', 'Cu_2.1C', 'Ni_2.1C', 'Cu_2.1T', 'Ni_2.1T', 'Dens_2', 'FM_2.1_A', 'Cu_2.1C_max', 'Cu_2.1C_min', 'Cu_2.1T_max','Cu_2.1T_min', 'Ni_rec']
column_2_2 = ['Cu_oreth', 'Ni_oreth', 'Ore_mass', 'Mass_2', 'Cu_resth', 'Ni_resth', 'Cu_2F', 'Ni_2F', 'Cu_2.2C', 'Ni_2.2C', 'Cu_2.2T', 'Ni_2.2T', 'Dens_2', 'FM_2.2_A', 'Cu_2.2C_max', 'Cu_2.2C_min', 'Cu_2.2T_max', 'Cu_2.2T_min', 'Ni_rec']
column_3_1 = ['Cu_oreth', 'Ni_oreth', 'Ore_mass', 'Cu_3F', 'Ni_3F', 'Cu_3.1C', 'Ni_3.1C', 'Cu_3.1T', 'Ni_3.1T', 'Dens_3', 'Mass_3', 'FM_3.1_A', 'Cu_3.1C_max', 'Cu_3.1C_min', 'Ni_3.1C_max', 'Ni_3.1C_min', 'Cu_3.1T_max', 'Cu_3.1T_min', 'Ni_rec']
column_3_2 = ['Cu_oreth', 'Ni_oreth', 'Ore_mass', 'Cu_3F', 'Ni_3F','Cu_3.2C', 'Ni_3.2C', 'Cu_3.2T', 'Ni_3.2T', 'Dens_3', 'Mass_3', 'FM_3.2_A', 'Cu_3.2C_max', 'Cu_3.2C_min', 'Ni_3.2C_max', 'Ni_3.2C_min', 'Cu_3.2T_max', 'Cu_3.2T_min', 'Ni_rec']
column_4_1 = ['Cu_oreth', 'Ni_oreth', 'Ore_mass', 'Dens_4', 'Mass_4', 'Vol_4', 'Cu_4F', 'Ni_4F', 'Ni_4.1C', 'Ni_4.1C_max', 'Ni_4.1C_min', 'Ni_4.1T', 'Ni_4.1T_max', 'Ni_4.1T_min', 'FM_4.1_A', 'Cu_resth', 'Ni_resth', 'Ni_rec']
column_4_2 = ['Cu_oreth', 'Ni_oreth', 'Ore_mass', 'Dens_4', 'Mass_4', 'Vol_4', 'Cu_4F', 'Ni_4F', 'Ni_4.2C', 'Ni_4.2C_max', 'Ni_4.2C_min', 'Ni_4.2T', 'Ni_4.2T_max', 'Ni_4.2T_min', 'FM_4.2_A', 'Cu_resth', 'Ni_resth', 'Ni_rec']
column_5_1 = ['Cu_oreth', 'Ni_oreth', 'Ore_mass', 'Dens_5', 'Mass_5', 'Vol_5', 'Ni_5F', 'Ni_5.1C', 'Ni_5.1C_max', 'Ni_5.1C_min', 'Ni_5.1T', 'Ni_5.1T_max', 'Ni_5.1T_min', 'FM_5.1_A', 'Cu_resth', 'Ni_resth', 'Ni_rec']
column_5_2 = ['Cu_oreth', 'Ni_oreth', 'Ore_mass', 'Dens_5', 'Mass_5', 'Vol_5', 'Ni_5F', 'Ni_5.2C', 'Ni_5.2C_max', 'Ni_5.2C_min', 'Ni_5.2T', 'Ni_5.2T_max', 'Ni_5.2T_min', 'FM_5.2_A', 'Cu_resth', 'Ni_resth', 'Ni_rec']
column_6_1 = ['Cu_oreth', 'Ni_oreth', 'Ore_mass', 'Dens_6', 'Mass_6', 'Vol_6', 'Ni_6F', 'Ni_6.1C', 'Ni_6.1C_max', 'Ni_6.1C_min', 'Ni_6.1T', 'Ni_6.1T_max', 'Ni_6.1T_min', 'FM_6.1_A', 'Cu_resth', 'Ni_resth', 'Ni_rec']
column_6_2 = ['Cu_oreth', 'Ni_oreth', 'Ore_mass', 'Dens_6', 'Mass_6', 'Vol_6', 'Ni_6F', 'Ni_6.2C', 'Ni_6.2C_max', 'Ni_6.2C_min', 'Ni_6.2T', 'Ni_6.2T_max', 'Ni_6.2T_min', 'FM_6.2_A', 'Cu_resth', 'Ni_resth', 'Ni_rec']

Анализ корреляции будем проводить методом Phik.

In [225]:
for list_columns in [column_1_1, column_1_2, column_2_1, column_2_2, column_3_1, column_3_2, column_4_1, column_4_2, column_5_1, column_5_2, column_6_1, column_6_2]:
  phik_overview = df_fill_median[list_columns].phik_matrix()
  phik_overview.round(2)

  plot_correlation_matrix(phik_overview.values,
                        x_labels=phik_overview.columns,
                        y_labels=phik_overview.index,
                        vmin=0, vmax=1, color_map="coolwarm",
                        title="Тепловая карта коэффициентов корреляции",
                        fontsize_factor=1,
                        figsize=(8, 8))
  plt.tight_layout()
  plt.show()


Output hidden; open in https://colab.research.google.com to view.

Из анализа видно:
1. Готовый выход меди и никеля мало зависит от установленных диапазонов. Корреляция лежит в диапазонах от 0,15 до 0,3.
2. Выход никеля, Ni_resth, больше зависит от Ni_1.1C_min и подобных столбцов, указывающих уставку. В меньшей степени он зависит от настоящей концентрации в смеси (столбцы типа Ni_1.1C).

## Подготовка данных для моделей

Так как в данных есть признаки, которые для обучения моделей вряд ли будут полезны, выделим их в отдельный датафрейм. Это признаки, указывающие на факт работы автоматического оптимизатора.

In [226]:
automate_column = ['FM_1.1_A', 'FM_1.2_A', 'FM_2.1_A', 'FM_2.2_A','FM_3.1_A', 'FM_3.2_A', 'FM_4.1_A','FM_4.2_A', 'FM_5.1_A', 'FM_5.2_A', 'FM_6.1_A', 'FM_6.2_A']

На основании анализа, проведенного выше, было видно, что есть как очень бедное сырье, так и очень богатое. Очевидно, что мы не сможем дать очень богатый выход, если на вход был подано бедное сырье. В этой целью введем в датафрейм новый признак - соотношение входа к выходу. При обучении моделей будем ориентироваться на него.

In [227]:
df_fill_median['ratio_Ni_in_out'] = df_fill_median['Ni_oreth'] / df_fill_median['Ni_resth']

<ipython-input-227-b5e377de50a1>:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_fill_median['ratio_Ni_in_out'] = df_fill_median['Ni_oreth'] / df_fill_median['Ni_resth']


У нас в данных есть установленные диапазоны даже в тех случаях, когда автомат был выключен. Очевидно, что это ошибочная информация. Анализ датафрейма показал, что только в 1277 случаях автомат работал на всех аппаратах. Чтобы сохранить данные и использовать весь датафрейм приравняем максимальную уставку для отключенных аппаратов к фактическим значениям металла в смеси. Для этого воспользуемся функциями, которые готовили ранее.

In [228]:
df_fill_median['Cu_3.2C_max'] = df_fill_median.apply(corr_range_Cu_3_2_C, axis=1)

In [229]:
df_fill_median['Cu_3.2T_max'] = df_fill_median.apply(corr_range_Cu_3_2_T, axis=1)

In [230]:
df_fill_median['Cu_3.1T_max'] = df_fill_median.apply(corr_range_Cu_3_1_T, axis=1)

In [231]:
df_fill_median['Cu_3.1C_max'] = df_fill_median.apply(corr_range_Cu_3_1_C, axis=1)

In [232]:
df_fill_median['Cu_2.2C_max'] = df_fill_median.apply(corr_range_Cu_2_2_C, axis=1)

In [233]:
df_fill_median['Cu_2.2T_max'] = df_fill_median.apply(corr_range_Cu_2_2_T, axis=1)

In [234]:
df_fill_median['Cu_2.1T_max'] = df_fill_median.apply(corr_range_Cu_2_1_T, axis=1)

In [235]:
df_fill_median['Cu_2.1C_max'] = df_fill_median.apply(corr_range_Cu_2_1_C, axis=1)

In [236]:
df_fill_median['Cu_1.1C_max'] = df_fill_median.apply(corr_range_Cu_1_1_C, axis=1)

In [237]:
df_fill_median['Cu_1.2C_max'] = df_fill_median.apply(corr_range_Cu_1_2_C, axis=1)

In [238]:
df_fill_median['Ni_6.1T_max'] = df_fill_median.apply(corr_range_Ni_6_1_T, axis=1)

In [239]:
df_fill_median['Ni_6.2T_max'] = df_fill_median.apply(corr_range_Ni_6_2_T, axis=1)

In [240]:
df_fill_median['Ni_6.2C_max'] = df_fill_median.apply(corr_range_Ni_6_2_C, axis=1)

In [241]:
df_fill_median['Ni_6.1C_max'] = df_fill_median.apply(corr_range_Ni_6_1_C, axis=1)

In [242]:
df_fill_median['Ni_5.1T_max'] = df_fill_median.apply(corr_range_Ni_5_1_T, axis=1)

In [243]:
df_fill_median['Ni_5.2T_max'] = df_fill_median.apply(corr_range_Ni_5_2_T, axis=1)

In [244]:
df_fill_median['Ni_5.2C_max'] = df_fill_median.apply(corr_range_Ni_5_2_C, axis=1)

In [245]:
df_fill_median['Ni_5.1C_max'] = df_fill_median.apply(corr_range_Ni_5_1_C, axis=1)

In [246]:
df_fill_median['Ni_4.1T_max'] = df_fill_median.apply(corr_range_Ni_4_1_T, axis=1)

In [247]:
df_fill_median['Ni_4.2T_max'] = df_fill_median.apply(corr_range_Ni_4_2_T, axis=1)

In [248]:
df_fill_median['Ni_4.2C_max'] = df_fill_median.apply(corr_range_Ni_4_2_C, axis=1)

In [249]:
df_fill_median['Ni_4.1C_max'] = df_fill_median.apply(corr_range_Ni_4_1_C, axis=1)

In [250]:
df_fill_median['Ni_1.1C_max'] = df_fill_median.apply(corr_range_Ni_1_1_c, axis=1)

In [251]:
df_fill_median['Ni_1.2C_max'] = df_fill_median.apply(corr_range_Ni_1_2_C, axis=1)

По условиях задачи нужно спрогнозировать минимальное и максимальное значения диапазона. Так как при прогнозировании может получиться, что минимальное значение будет выше максимального, будем прогнозировать только максимальное. А минимальное значение получим расчетным путем.

In [252]:
columns_test_max = ['Ni_1.1C_max', 'Cu_1.1C_max',
'Ni_1.2C_max', 'Cu_1.2C_max',
 'Cu_2.1T_max', 'Cu_2.2T_max',
 'Cu_3.1T_max', 'Cu_3.2T_max',
 'Ni_4.1T_max', 'Ni_4.1C_max',
'Ni_4.2T_max', 'Ni_4.2C_max',
 'Ni_5.1T_max',  'Ni_5.1C_max',
 'Ni_5.2T_max', 'Ni_5.2C_max',
 'Ni_6.1T_max', 'Ni_6.1C_max',
'Ni_6.2T_max', 'Ni_6.2C_max']

Сохраним тестовые столбцы в отдельный список, чтобы использовать их при обучении моделей.

In [253]:
columns_test = df_test.columns

## Обучение моделей

Для прогнозирования признаков будем использовать метод градиентного бустинга. Для возможности выбора лучшего, возьмем два возможных варианта: XGBoost и LightGBM. Прогнозировать будем сразу 20 значений с помощью функции MultiOutputRegressor. Для обучения будем использовать датафрейм, в котором пропуски заполнены медианой.

Подготовим выборки:

In [254]:
X = df_fill_median.drop(columns_test, axis=1).drop(automate_column, axis=1)
y = df_fill_median[columns_test_max]

scaler = StandardScaler()


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle = False)

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Подготовим и обучим XGBoost

In [255]:
xgb = XGBRegressor(objective='reg:squarederror', random_state=42)
multi_regressor_xgb = MultiOutputRegressor(xgb)

In [256]:
multi_regressor_xgb.fit(X_train, y_train)

MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, ...))

Получим прогноз на тестовой выборке

In [257]:
y_pred_XG = multi_regressor_xgb.predict(X_test)

Оценим качество модели по двум метрикам.

In [258]:
r2 = r2_score(y_test, y_pred_XG)
rmse = root_mean_squared_error(y_test, y_pred_XG)
print(f'R2 Score (XGBoost): {r2}')
print(f'RMSE (XGBoost): {rmse}')

R2 Score (XGBoost): -1.8124030828475952
RMSE (XGBoost): 1.3584225141402546


Теперь подготовим и обучим LightGBM

In [259]:
lgb = LGBMRegressor(random_state=42)
multi_regressor_lgb = MultiOutputRegressor(lgb)

In [260]:
multi_regressor_lgb.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.029074 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16864
[LightGBM] [Info] Number of data points in the train set: 24268, number of used features: 72
[LightGBM] [Info] Start training from score 1.276482
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.023228 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16864
[LightGBM] [Info] Number of data points in the train set: 24268, number of used features: 72
[LightGBM] [Info] Start training from score 2.040451
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.023668 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16864
[LightGBM] [Info] Number of data points in the train set: 24268, number of used features: 72
[LightGBM] [Info] Start 

MultiOutputRegressor(estimator=LGBMRegressor(random_state=42))

Получим пргноз на тестовых данных

In [261]:
y_pred_L = multi_regressor_lgb.predict(X_test)

Оценим качество модели.

In [262]:
r2 = r2_score(y_test, y_pred_L)
rmse = root_mean_squared_error(y_test, y_pred_L)
print(f'R2 Score (LightGBM): {r2}')
print(f'RMSE (LightGBM): {rmse}')


R2 Score (LightGBM): -0.4850933394473933
RMSE (LightGBM): 1.2415782950418741


Метрика невысокая, но лучшее качество получилось на LightGBM. Его и будем использовать.

## Получение прогноза на тестовых данных

Для того, чтобы получить прогноз на тестовых данных, необходимо в нужный датафрейм добавить новые признаки. Сделаем это.

In [263]:
feature_for_test = df_fill_median.drop(columns_test, axis=1).drop(automate_column, axis=1)

На основании анализа было установлено, что самое частое соотношение вход/выход равняется 0,184810. Его и установим как константу для всего датафрейма в столбце ratio_Ni_in_out.

In [264]:
df_predict = df_test.merge(feature_for_test, on='MEAS_DT', how='left')

In [265]:
df_predict.loc[:,'ratio_Ni_in_out'] = 0.184810

In [266]:
df_predict.head()

,Ni_1.1C_min,Ni_1.1C_max,Cu_1.1C_min,Cu_1.1C_max,Ni_1.2C_min,Ni_1.2C_max,Cu_1.2C_min,Cu_1.2C_max,Cu_2.1T_min,Cu_2.1T_max,...,Ni_6.2T,Ni_6F,Ni_oreth,Ni_rec,Ni_resth,Ore_mass,Vol_4,Vol_5,Vol_6,ratio_Ni_in_out
MEAS_DT,,,,,,,,,,,,,,,,,,,,,
2024-01-19 12:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0752,8.99060,1.7944,0.974712,8.7576,1268.5,1476.09375,996.7500,630.65625,0.18481
2024-01-19 12:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0185,8.99835,1.7814,0.973093,8.9480,1034.5,1488.59375,989.7500,633.18750,0.18481
2024-01-19 12:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0600,9.01330,1.7346,0.945329,8.8334,1160.5,1482.96875,979.8125,639.21875,0.18481
2024-01-19 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0595,8.96760,1.6932,0.937162,8.7097,1213.5,1485.06250,992.5625,641.75000,0.18481
2024-01-19 13:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0414,8.99295,1.6646,0.940773,8.8970,1270.0,1489.21875,987.3125,606.53125,0.18481


In [267]:
df_predict = df_predict.drop(columns_test, axis=1)

Датафрейм для прогноза подготовлен. Масштабируем его и получим предсказания.

In [268]:
df_predict = scaler.transform(df_predict)

In [269]:
test_predict = multi_regressor_lgb.predict(df_predict)

Соберем предсказания в датафрейм

In [270]:
test_predict = pd.DataFrame(data=test_predict, columns=columns_test_max)

In [271]:
test_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6740 entries, 0 to 6739
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Ni_1.1C_max  6740 non-null   float64
 1   Cu_1.1C_max  6740 non-null   float64
 2   Ni_1.2C_max  6740 non-null   float64
 3   Cu_1.2C_max  6740 non-null   float64
 4   Cu_2.1T_max  6740 non-null   float64
 5   Cu_2.2T_max  6740 non-null   float64
 6   Cu_3.1T_max  6740 non-null   float64
 7   Cu_3.2T_max  6740 non-null   float64
 8   Ni_4.1T_max  6740 non-null   float64
 9   Ni_4.1C_max  6740 non-null   float64
 10  Ni_4.2T_max  6740 non-null   float64
 11  Ni_4.2C_max  6740 non-null   float64
 12  Ni_5.1T_max  6740 non-null   float64
 13  Ni_5.1C_max  6740 non-null   float64
 14  Ni_5.2T_max  6740 non-null   float64
 15  Ni_5.2C_max  6740 non-null   float64
 16  Ni_6.1T_max  6740 non-null   float64
 17  Ni_6.1C_max  6740 non-null   float64
 18  Ni_6.2T_max  6740 non-null   float64
 19  Ni_6.2

Добавим дату в датафрейм с прогнозом.

In [272]:
test_predict = test_predict.join(df_test_index['MEAS_DT'])

In [273]:
test_predict.head()

,Ni_1.1C_max,Cu_1.1C_max,Ni_1.2C_max,Cu_1.2C_max,Cu_2.1T_max,Cu_2.2T_max,Cu_3.1T_max,Cu_3.2T_max,Ni_4.1T_max,Ni_4.1C_max,...,Ni_4.2C_max,Ni_5.1T_max,Ni_5.1C_max,Ni_5.2T_max,Ni_5.2C_max,Ni_6.1T_max,Ni_6.1C_max,Ni_6.2T_max,Ni_6.2C_max,MEAS_DT
0,0.126030,0.229129,-0.154983,-0.017751,0.439897,0.387716,2.068924,1.197383,-0.014005,-0.065526,...,-0.061361,-0.032811,-0.096093,-0.032444,-0.490390,0.083186,0.726222,0.092828,0.318938,2024-01-19 12:15:00
1,0.151898,0.241396,0.062408,0.266929,0.441138,0.388381,1.891964,1.130388,-0.026110,-0.122456,...,-0.148093,-0.040162,-0.289241,-0.043725,-0.613606,0.166249,2.491284,0.044000,2.805763,2024-01-19 12:30:00
2,0.171698,0.038552,-0.058532,0.167683,0.439454,0.389808,1.434390,1.108458,-0.031005,-0.162530,...,-0.103298,-0.025991,-0.138890,-0.036990,-0.486490,-0.001033,-0.345021,0.012813,-0.678333,2024-01-19 12:45:00
3,0.191227,0.439367,-0.015633,0.096750,0.439281,0.389912,1.027498,1.059363,-0.019813,-0.185175,...,-0.100762,0.025224,-0.103661,-0.029575,-0.324124,0.206314,2.337442,0.115581,2.470028,2024-01-19 13:00:00
4,-0.056396,0.059285,-0.077815,0.227234,0.442114,0.391841,0.902858,1.005033,-0.015747,-0.220190,...,-0.057681,-0.031926,-0.356521,-0.043492,-0.596838,-0.055703,-0.678371,-0.004868,-0.492539,2024-01-19 13:15:00


Датафрейм получен, но там только максимальные значения. Рассчитаем минимальные:

In [274]:
coef_Ni_1_1C = describe.loc['50%', 'Ni_1.1C_max'] / describe.loc['50%', 'Ni_1.1C_min']

In [275]:
test_predict['Ni_1.1C_min'] = test_predict['Ni_1.1C_max'] / coef_Ni_1_1C

In [276]:
coef_Cu_1_1C = describe.loc['50%', 'Cu_1.1C_max'] / describe.loc['50%', 'Cu_1.1C_min']

In [277]:
test_predict['Cu_1.1C_min'] = test_predict['Cu_1.1C_max'] / coef_Cu_1_1C

In [278]:
coef_Ni_1_2C = describe.loc['50%', 'Ni_1.2C_max'] / describe.loc['50%', 'Ni_1.2C_min']

In [279]:
test_predict['Ni_1.2C_min'] = test_predict['Ni_1.2C_max'] / coef_Ni_1_2C

In [280]:
coef_Cu_1_2C = describe.loc['50%', 'Cu_1.2C_max'] / describe.loc['50%', 'Cu_1.2C_min']

In [281]:
test_predict['Cu_1.2C_min'] = test_predict['Cu_1.2C_max'] / coef_Cu_1_2C

In [282]:
coef_Cu_2_1T = describe.loc['50%', 'Cu_2.1T_max'] / describe.loc['50%', 'Cu_2.1T_min']

In [283]:
test_predict['Cu_2.1T_min'] = test_predict['Cu_2.1T_max'] / coef_Cu_2_1T

In [284]:
coef_Cu_2_2T = describe.loc['50%', 'Cu_2.2T_max'] / describe.loc['50%', 'Cu_2.2T_min']

In [285]:
test_predict['Cu_2.2T_min'] = test_predict['Cu_2.2T_max'] / coef_Cu_2_2T

In [286]:
coef_Cu_3_1T = describe.loc['50%', 'Cu_3.1T_max'] / describe.loc['50%', 'Cu_3.1T_min']

In [287]:
test_predict['Cu_3.1T_min'] = test_predict['Cu_3.1T_max'] / coef_Cu_3_1T

In [288]:
coef_Cu_3_2T = describe.loc['50%', 'Cu_3.2T_max'] / describe.loc['50%', 'Cu_3.2T_min']

In [289]:
test_predict['Cu_3.2T_min'] = test_predict['Cu_3.2T_max'] / coef_Cu_3_2T

In [290]:
coef_Ni_4_1T = describe.loc['50%', 'Ni_4.1T_max'] / describe.loc['50%', 'Ni_4.1T_min']

In [291]:
test_predict['Ni_4.1T_min'] = test_predict['Ni_4.1T_max'] / coef_Ni_4_1T

In [292]:
coef_Ni_4_1C = describe.loc['50%', 'Ni_4.1C_max'] / describe.loc['50%', 'Ni_4.1C_min']

In [293]:
test_predict['Ni_4.1C_min'] = test_predict['Ni_4.1C_max'] / coef_Ni_4_1C

In [294]:
coef_Ni_4_2T = describe.loc['50%', 'Ni_4.2T_max'] / describe.loc['50%', 'Ni_4.2T_min']

In [295]:
test_predict['Ni_4.2T_min'] = test_predict['Ni_4.2T_max'] / coef_Ni_4_2T

In [296]:
coef_Ni_4_2C = describe.loc['50%', 'Ni_4.2C_max'] / describe.loc['50%', 'Ni_4.2C_min']

In [297]:
test_predict['Ni_4.2C_min'] = test_predict['Ni_4.2C_max'] / coef_Ni_4_2C

In [298]:
coef_Ni_5_1T = describe.loc['50%', 'Ni_5.1T_max'] / describe.loc['50%', 'Ni_5.1T_min']

In [299]:
test_predict['Ni_5.1T_min'] = test_predict['Ni_5.1T_max'] / coef_Ni_5_1T

In [300]:
coef_Ni_5_1C = describe.loc['50%', 'Ni_5.1C_max'] / describe.loc['50%', 'Ni_5.1C_min']

In [301]:
test_predict['Ni_5.1C_min'] = test_predict['Ni_5.1C_max'] / coef_Ni_5_1C

In [302]:
coef_Ni_5_2T = describe.loc['50%', 'Ni_5.2T_max'] / describe.loc['50%', 'Ni_5.2T_min']

In [303]:
test_predict['Ni_5.2T_min'] = test_predict['Ni_5.2T_max'] / coef_Ni_5_2T

In [304]:
coef_Ni_5_2C = describe.loc['50%', 'Ni_5.2C_max'] / describe.loc['50%', 'Ni_5.2C_min']

In [305]:
test_predict['Ni_5.2C_min'] = test_predict['Ni_5.2C_max'] / coef_Ni_5_2C

In [306]:
coef_Ni_6_1T = describe.loc['50%', 'Ni_6.1T_max'] / describe.loc['50%', 'Ni_6.1T_min']

In [307]:
test_predict['Ni_6.1T_min'] = test_predict['Ni_6.1T_max'] / coef_Ni_6_1T

In [308]:
coef_Ni_6_1C = describe.loc['50%', 'Ni_6.1C_max'] / describe.loc['50%', 'Ni_6.1C_min']

In [309]:
test_predict['Ni_6.1C_min'] = test_predict['Ni_6.1C_max'] / coef_Ni_6_1C

In [310]:
coef_Ni_6_2T = describe.loc['50%', 'Ni_6.2T_max'] / describe.loc['50%', 'Ni_6.2T_min']

In [311]:
test_predict['Ni_6.2T_min'] = test_predict['Ni_6.2T_max'] / coef_Ni_6_2T

In [312]:
coef_Ni_6_2C = describe.loc['50%', 'Ni_6.2C_max'] / describe.loc['50%', 'Ni_6.2C_min']

In [313]:
test_predict['Ni_6.2C_min'] = test_predict['Ni_6.2C_max'] / coef_Ni_6_2C

In [314]:
test_predict.head()

,Ni_1.1C_max,Cu_1.1C_max,Ni_1.2C_max,Cu_1.2C_max,Cu_2.1T_max,Cu_2.2T_max,Cu_3.1T_max,Cu_3.2T_max,Ni_4.1T_max,Ni_4.1C_max,...,Ni_4.2T_min,Ni_4.2C_min,Ni_5.1T_min,Ni_5.1C_min,Ni_5.2T_min,Ni_5.2C_min,Ni_6.1T_min,Ni_6.1C_min,Ni_6.2T_min,Ni_6.2C_min
0,0.126030,0.229129,-0.154983,-0.017751,0.439897,0.387716,2.068924,1.197383,-0.014005,-0.065526,...,-0.033005,-0.045694,-0.025780,-0.084952,-0.023792,-0.447748,0.072095,0.695318,0.080451,0.312152
1,0.151898,0.241396,0.062408,0.266929,0.441138,0.388381,1.891964,1.130388,-0.026110,-0.122456,...,-0.040334,-0.110282,-0.031556,-0.255705,-0.032065,-0.560249,0.144083,2.385272,0.038133,2.746066
2,0.171698,0.038552,-0.058532,0.167683,0.439454,0.389808,1.434390,1.108458,-0.031005,-0.162530,...,-0.036981,-0.076924,-0.020421,-0.122787,-0.027126,-0.444186,-0.000895,-0.330340,0.011105,-0.663901
3,0.191227,0.439367,-0.015633,0.096750,0.439281,0.389912,1.027498,1.059363,-0.019813,-0.185175,...,0.012970,-0.075036,0.019819,-0.091642,-0.021689,-0.295939,0.178806,2.237977,0.100170,2.417475
4,-0.056396,0.059285,-0.077815,0.227234,0.442114,0.391841,0.902858,1.005033,-0.015747,-0.220190,...,0.006675,-0.042954,-0.025085,-0.315185,-0.031894,-0.544939,-0.048276,-0.649504,-0.004219,-0.482059


В датафрейме получились отрицательные значения, которых быть не должно. Приравняем их к 0. При работе предполагается, что в этом случае автомат на этом аппарате будет выключен.

In [315]:
for column in test_predict.drop('MEAS_DT', axis=1).columns:
  test_predict[column] = test_predict[column].apply(delete_zero)


In [316]:
test_predict.head()

,Ni_1.1C_max,Cu_1.1C_max,Ni_1.2C_max,Cu_1.2C_max,Cu_2.1T_max,Cu_2.2T_max,Cu_3.1T_max,Cu_3.2T_max,Ni_4.1T_max,Ni_4.1C_max,...,Ni_4.2T_min,Ni_4.2C_min,Ni_5.1T_min,Ni_5.1C_min,Ni_5.2T_min,Ni_5.2C_min,Ni_6.1T_min,Ni_6.1C_min,Ni_6.2T_min,Ni_6.2C_min
0,0.126030,0.229129,0.000000,0.000000,0.439897,0.387716,2.068924,1.197383,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.072095,0.695318,0.080451,0.312152
1,0.151898,0.241396,0.062408,0.266929,0.441138,0.388381,1.891964,1.130388,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.144083,2.385272,0.038133,2.746066
2,0.171698,0.038552,0.000000,0.167683,0.439454,0.389808,1.434390,1.108458,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.011105,0.000000
3,0.191227,0.439367,0.000000,0.096750,0.439281,0.389912,1.027498,1.059363,0.0,0.0,...,0.012970,0.0,0.019819,0.0,0.0,0.0,0.178806,2.237977,0.100170,2.417475
4,0.000000,0.059285,0.000000,0.227234,0.442114,0.391841,0.902858,1.005033,0.0,0.0,...,0.006675,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000


Нулевые значения обработаны. Теперь расставим столбцы в порядке, в котором они были изначально в тестовом датафрейме.

In [317]:
test_predict = test_predict.loc[:, ['MEAS_DT', 'Ni_1.1C_min', 'Ni_1.1C_max', 'Cu_1.1C_min', 'Cu_1.1C_max',
       'Ni_1.2C_min', 'Ni_1.2C_max', 'Cu_1.2C_min', 'Cu_1.2C_max',
       'Cu_2.1T_min', 'Cu_2.1T_max', 'Cu_2.2T_min', 'Cu_2.2T_max',
       'Cu_3.1T_min', 'Cu_3.1T_max', 'Cu_3.2T_min', 'Cu_3.2T_max',
       'Ni_4.1T_min', 'Ni_4.1T_max', 'Ni_4.1C_min', 'Ni_4.1C_max',
       'Ni_4.2T_min', 'Ni_4.2T_max', 'Ni_4.2C_min', 'Ni_4.2C_max',
       'Ni_5.1T_min', 'Ni_5.1T_max', 'Ni_5.1C_min', 'Ni_5.1C_max',
       'Ni_5.2T_min', 'Ni_5.2T_max', 'Ni_5.2C_min', 'Ni_5.2C_max',
       'Ni_6.1T_min', 'Ni_6.1T_max', 'Ni_6.1C_min', 'Ni_6.1C_max',
       'Ni_6.2T_min', 'Ni_6.2T_max', 'Ni_6.2C_min', 'Ni_6.2C_max']]

In [318]:
test_predict.head()

,MEAS_DT,Ni_1.1C_min,Ni_1.1C_max,Cu_1.1C_min,Cu_1.1C_max,Ni_1.2C_min,Ni_1.2C_max,Cu_1.2C_min,Cu_1.2C_max,Cu_2.1T_min,...,Ni_5.2C_min,Ni_5.2C_max,Ni_6.1T_min,Ni_6.1T_max,Ni_6.1C_min,Ni_6.1C_max,Ni_6.2T_min,Ni_6.2T_max,Ni_6.2C_min,Ni_6.2C_max
0,2024-01-19 12:15:00,0.098461,0.126030,0.205748,0.229129,0.000000,0.000000,0.000000,0.000000,0.345634,...,0.0,0.0,0.072095,0.083186,0.695318,0.726222,0.080451,0.092828,0.312152,0.318938
1,2024-01-19 12:30:00,0.118670,0.151898,0.216764,0.241396,0.054607,0.062408,0.236129,0.266929,0.346609,...,0.0,0.0,0.144083,0.166249,2.385272,2.491284,0.038133,0.044000,2.746066,2.805763
2,2024-01-19 12:45:00,0.134139,0.171698,0.034619,0.038552,0.000000,0.000000,0.148335,0.167683,0.345286,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.011105,0.012813,0.000000,0.000000
3,2024-01-19 13:00:00,0.149396,0.191227,0.394534,0.439367,0.000000,0.000000,0.085587,0.096750,0.345150,...,0.0,0.0,0.178806,0.206314,2.237977,2.337442,0.100170,0.115581,2.417475,2.470028
4,2024-01-19 13:15:00,0.000000,0.000000,0.053235,0.059285,0.000000,0.000000,0.201015,0.227234,0.347375,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## Реализация ограничений

Как было сказанно в начале, есть ряд ограничений. А именно: обновление не чаще 1 раза в два часа и совпадение количество знаков после запятых. Реализуем их в цикле с применением функции.

In [319]:
for column in test_predict.drop('MEAS_DT', axis=1).columns:
  def calculate_rolling_mean(df, group_size):
    df['group'] = (df.index // group_size)
    means = df.groupby('group')[column].transform('mean')
    df[column] = round(means, 1)

    return df

# Вызываем функцию с необходимой группировкой по 8 строк
  group_size = 8
  test_predict = calculate_rolling_mean(test_predict, group_size)



In [320]:
test_predict.head()

,MEAS_DT,Ni_1.1C_min,Ni_1.1C_max,Cu_1.1C_min,Cu_1.1C_max,Ni_1.2C_min,Ni_1.2C_max,Cu_1.2C_min,Cu_1.2C_max,Cu_2.1T_min,...,Ni_5.2C_max,Ni_6.1T_min,Ni_6.1T_max,Ni_6.1C_min,Ni_6.1C_max,Ni_6.2T_min,Ni_6.2T_max,Ni_6.2C_min,Ni_6.2C_max,group
0,2024-01-19 12:15:00,0.1,0.1,0.2,0.2,0.0,0.0,0.1,0.1,0.3,...,0.0,0.1,0.1,0.7,0.7,0.0,0.0,0.7,0.7,0
1,2024-01-19 12:30:00,0.1,0.1,0.2,0.2,0.0,0.0,0.1,0.1,0.3,...,0.0,0.1,0.1,0.7,0.7,0.0,0.0,0.7,0.7,0
2,2024-01-19 12:45:00,0.1,0.1,0.2,0.2,0.0,0.0,0.1,0.1,0.3,...,0.0,0.1,0.1,0.7,0.7,0.0,0.0,0.7,0.7,0
3,2024-01-19 13:00:00,0.1,0.1,0.2,0.2,0.0,0.0,0.1,0.1,0.3,...,0.0,0.1,0.1,0.7,0.7,0.0,0.0,0.7,0.7,0
4,2024-01-19 13:15:00,0.1,0.1,0.2,0.2,0.0,0.0,0.1,0.1,0.3,...,0.0,0.1,0.1,0.7,0.7,0.0,0.0,0.7,0.7,0


Удалим вспомогательный столбец.

In [321]:
 test_predict =  test_predict.drop('group', axis=1)

In [322]:
 test_predict.head()

,MEAS_DT,Ni_1.1C_min,Ni_1.1C_max,Cu_1.1C_min,Cu_1.1C_max,Ni_1.2C_min,Ni_1.2C_max,Cu_1.2C_min,Cu_1.2C_max,Cu_2.1T_min,...,Ni_5.2C_min,Ni_5.2C_max,Ni_6.1T_min,Ni_6.1T_max,Ni_6.1C_min,Ni_6.1C_max,Ni_6.2T_min,Ni_6.2T_max,Ni_6.2C_min,Ni_6.2C_max
0,2024-01-19 12:15:00,0.1,0.1,0.2,0.2,0.0,0.0,0.1,0.1,0.3,...,0.0,0.0,0.1,0.1,0.7,0.7,0.0,0.0,0.7,0.7
1,2024-01-19 12:30:00,0.1,0.1,0.2,0.2,0.0,0.0,0.1,0.1,0.3,...,0.0,0.0,0.1,0.1,0.7,0.7,0.0,0.0,0.7,0.7
2,2024-01-19 12:45:00,0.1,0.1,0.2,0.2,0.0,0.0,0.1,0.1,0.3,...,0.0,0.0,0.1,0.1,0.7,0.7,0.0,0.0,0.7,0.7
3,2024-01-19 13:00:00,0.1,0.1,0.2,0.2,0.0,0.0,0.1,0.1,0.3,...,0.0,0.0,0.1,0.1,0.7,0.7,0.0,0.0,0.7,0.7
4,2024-01-19 13:15:00,0.1,0.1,0.2,0.2,0.0,0.0,0.1,0.1,0.3,...,0.0,0.0,0.1,0.1,0.7,0.7,0.0,0.0,0.7,0.7


Прогноз получен. Сохраним данные в отдельную таблицу.

In [323]:
test_predict.to_csv('test.csv', index=False)

## Выводы
В рамках работы выполнен исследовательский и корреляционный анализ данных, а также выполнено прогнозирование диапазонов. Лучшей моделью оказалась LightGBM. Однако метрика получилась невысокая. Улучшение метрики возможно более тщательной подготовкой данных и добавляем новых признаков.
